# VnV 

- JPark


In [1]:

#------------------------------------------------------
# Config
#------------------------------------------------------

# Test images

zip_images_url = 'http://keticmr.iptime.org:22080/edgeai/images/imagenet-mini-val.zip'
zip_images = 'imagenet-mini-val.zip'
dataset_root = './dataset'
fpath_zip_images = dataset_root + '/' + zip_images
fpath_testimages = dataset_root + '/imagenet-mini-val/'

# Models
#names = ['res18_weights.pth', 'res34_weights.pth', 'res50_weights.pth', 'res101_weights.pth', 'res152_weights.pth']
names = ['resnet18-dict.pth', 'resnet34-dict.pth', 'resnet50-dict.pth', 'resnet101-dict.pth', 'resnet152-dict.pth']
urlroot = 'http://keticmr.iptime.org:22080/edgeai/models_jpark/'
modeldir = './checkpoint/'

urlmodels = []
for name in names:
    urlmodels.append( urlroot + name )




In [ ]:

#------------------------------------------------------
# Download data
#------------------------------------------------------
import os
# make download directory
def makedir(path): 
    try: 
        os.makedirs(path)
    except OSError: 
        if not os.path.isdir(path): 
            raise
    return os.path.abspath(path)

# Download images
import urllib, os

d = makedir(dataset_root) # 저장 공간 생성
url, fname = (zip_images_url, fpath_zip_images)
try: 
    urllib.URLopener().retrieve(url, fname)
except: 
    urllib.request.urlretrieve(url, fname)
    

# Unzip
cmd = 'unzip ' + fpath_zip_images + ' -d ' + dataset_root
print(cmd)
os.system(cmd)

# download models
makedir(modeldir)

for name in names:
    torch.hub.download_url_to_file(urlroot+name, modeldir+name)
    

## Run

In [2]:
#------------------------------------------------------
# Import packages
#------------------------------------------------------
import torch


model_fpaths = []
for name in names:
    model_fpaths.append(modeldir + name)

model_fpaths
#print(model_fpaths)
#print(urlroot+name)
#print(modeldir+name)


#------------------------------------------------------
# initial models
#------------------------------------------------------

model_names = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']
models = []

for model_name in model_names:
    m = torch.hub.load('pytorch/vision:v0.10.0', model_name, pretrained=False)
    models.append(m)
    m.eval()
    
# load saved weigts to the initial model
for idx, model in enumerate(models):
    checkpoint = urlmodels[idx]
    model.load_state_dict(torch.hub.load_state_dict_from_url(checkpoint, progress=False))
    model.eval()
    print(idx)
    
    

Using cache found in /Users/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/jpark/.cache/torch/hub/pytorch_vision_v0.10.0


0
1
2
3
4


## Test data

In [3]:
from glob import iglob

'''
# read test files
testfiles = []
for fname in sorted( iglob(fpath_testimages + '**/*.JPEG', recursive=True) ):
    testfiles.append(fname)
'''

idx_gt = []
idx = 0
testfiles = []
for d in sorted( iglob(fpath_testimages + 'n*', recursive=False) ):
    for fname in sorted( iglob(d + '/*.JPEG', recursive=True) ):
        testfiles.append(fname)
        idx_gt.append( idx )
    idx += 1


# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]




In [4]:
model_names

['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']

In [8]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
import time
from tqdm import tqdm
import numpy as np


'''
# Define transforms for the evaluation phase
preprocess = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])
'''

for i, model in enumerate(models):
    print(model_names[i])
    
    top1_cnt = 0
    top5_cnt = 0

    testset = testfiles[:100]
    n = len(testset)
    idx = 0
    
    start = time.time()


    preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    for fpath in tqdm( testset ):
        #print( fpath )
        input_image = Image.open(fpath)

        try:
            input_tensor = preprocess(input_image)
        except:
            #print(idx)
            # gray scale to color
            input_image = Image.open(fpath).convert("RGB")
            input_tensor = preprocess(input_image)

        input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

        # move the input and model to GPU for speed if available
        if torch.cuda.is_available():
            input_batch = input_batch.to('cuda')
            model.to('cuda')
        else:
            model.to('cpu')
    

        with torch.no_grad():
            output = model(input_batch)

        # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
        #print(output[0])

        # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        #print(probabilities)

    
        # Show top categories per image
        top5_prob, top5_catid = torch.topk(probabilities, 5)
        for i in range(top5_prob.size(0)):
            #print(top5_catid[i])
            #pass
            #print(idx, ' ', categories[top5_catid[i]], top5_prob[i].item())

            if( top5_catid[i] == idx_gt[idx] ):
                top1_cnt += 1

        #print('')
        idx += 1

    end = time.time()
    print('n = ', n)
    print('top1_cnt = ', top1_cnt)
    print('top1_cnt/n = ', top1_cnt/n)
    print('time = ', end - start)

resnet18


100%|█████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.62it/s]


n =  100
top1_cnt =  98
top1_cnt/n =  0.98
time =  6.4047300815582275
resnet34


100%|█████████████████████████████████████████████████████████████| 100/100 [00:11<00:00,  9.04it/s]


n =  100
top1_cnt =  98
top1_cnt/n =  0.98
time =  11.058824300765991
resnet50


...

n =  100
top1_cnt =  99
top1_cnt/n =  0.99
time =  13.026250123977661
resnet101


100%|█████████████████████████████████████████████████████████████| 100/100 [00:21<00:00,  4.58it/s]


n =  100
top1_cnt =  100
top1_cnt/n =  1.0
time =  21.852193117141724
resnet152


100%|█████████████████████████████████████████████████████████████| 100/100 [00:30<00:00,  3.26it/s]

n =  100
top1_cnt =  99
top1_cnt/n =  0.99
time =  30.642608880996704


In [ ]:
n =  3923
top1_cnt =  3519
top1_cnt/n =  0.8970175885801682
time =  99.26183104515076

In [ ]:
n =  3923
top1_cnt =  3299
top1_cnt/n =  0.8409380576089728
time =  37.87246131896973

In [ ]:
resnet18

n =  3923
top1_cnt =  3482
top1_cnt/n =  0.887586031098649
time =  429.7253770828247